In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# 修正したモデルから卒論時に集計したデータを作成する

1. 表

| ベンチマーク名 | 平均誤差率(%) | コスト比(%) |
|---------|----------|---------|


2. 表

| ベンチマーク名 | 採用割合(最大MAPE(%), 最小MAPE(%)) |
|---------|----------------------------|
|         | モデル(1), モデル(2), ...        |


* 平均誤差率：大規模実行時の関数コール回数との比較
* MAPE：トレーニングデータとの比較

In [ ]:
plt.figure(figsize=(4, 3))
plt.xlabel("使用したプロファイル数")
plt.ylabel("平均誤差率(%)")

In [ ]:

benchmark = "mg"
fix = "Class"
if benchmark == "bt" or benchmark == "sp":
    processes = processes_onlyBTSP
else:
    processes = processes_excludeBTSP
benchmarkClasses = ["A", "B", "C", "D"]
fixedBenchmarkClass = "B"
fixedProcess = 64

# 引数の条件に合った生の実験データを取得する
DF = returnRawDFperBenchmark(Benchmark=benchmark, fix=fix, benchmarkClass=benchmarkClasses,
                             FixedProcess=fixedProcess, Processes=processes, FixedBenchmarkClass=fixedBenchmarkClass)
# 取得した生の実験データから NaN が含まれる関数の実験データを削除
noNaNDF = DF.dropna(how='any')


# 必要な変数一覧
# process：リスト
# functinName：文字列
# 対象となるプロセス数を格納した変数
# 対象となる問題サイズを格納した変数
# benchmark：文字列
# fix：文字列(Process or Class)

# 取得した実験データから NaN が含まれない関数名のリスト
functionNames = noNaNDF.index.tolist()
# プロセス数のリスト
processes = noNaNDF.columns.tolist()
# 集計するためのDF
calculatedDF = pd.DataFrame(columns=return_numOfColumns())
calculatedDF = calculatedDF.astype(return_numOfColumns(dataType=True))


for functionName in functionNames:
    for i in reversed(range(1, len(processes))):

        indexSeparator = i+1
        raw_x = noNaNDF.loc[functionName].tolist()[:indexSeparator]
        raw_y = processes[:indexSeparator]
        targetNumOfProcess = 256
        targetProblemSize = fixedBenchmarkClass
        dataSeries = returnSeriesOfData(benchmarkName=benchmark, functionName=functionName, raw_x=raw_x, raw_y=raw_y,
                                        fix=fix, fixed=fixedBenchmarkClass, targetProcess=targetNumOfProcess, targetProblemSize=targetProblemSize)
        calculatedDF = calculatedDF.append(dataSeries, ignore_index=True)
calculatedDF

In [ ]:
DF = returnRawDFperBenchmark(Benchmark="mg", fix="Process", benchmarkClass=["A", "B", "C", "D"], Processes=[
                             1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="B", FixedProcess=64)
DF.dropna(how='any')
DF